In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
# from transformers import AutoImageProcessor, ResNetForImageClassification,ResNetConfig
import torch
from abc import ABC,abstractmethod

In [36]:
#TODO check if layers are actually being trained
#TODO add more models
#TODO compare metrics
#TODO test out prediction loop
#TODO learn how to use tensorboard

#TODO week2 Balance Dataset
#TODO day2 Implement checkpointing and checkpoint loading
#TODO day2 implement an actually good and modular pipeline
#TODO day2 EDA + explore augments
#TODO day2 normalize and test out mean/median/std of pixel values
#TODO day2 Actually implement additional layers and finish what i started
#TODO Verify data is sent correctly (esp labels)

#TODO day2 implement localization/segmentation + research more abt osteopenia
#interesting models: yolo v5, medsam

###DATASET PREP###

In [37]:
# projectPath = r"OsteoBinary"
# pathList = []
# labelList = []
# dirList = os.listdir(projectPath)[:3]
# for idx, x in enumerate(dirList):
#     for xx in os.listdir(f"{projectPath}/{x}"):
#         pathList.append(f"{projectPath}/{x}/{xx}")
#         labelList.append(idx)

In [38]:
projectPath = r"C:\Users\assaw\Documents\c_stuff\Python\machine learning\ISAIConference\testIm"
pathList = []
labelList = []
dirList = os.listdir(projectPath)[:3]
for idx, x in enumerate(dirList):
    for xx in os.listdir(f"{projectPath}/{x}"):
        pathList.append(f"{projectPath}/{x}/{xx}")
        labelList.append(idx)
        
# dir structure: projectPath/label/image

In [39]:
pathList

['C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N10.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N100.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N101.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N11.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N110.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N111.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N120.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N121.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\\Python\\machine learning\\ISAIConference\\testIm/normal/N130.jpg',
 'C:\\Users\\assaw\\Documents\\c_stuff\

In [40]:
for idx,x in enumerate(dirList):
    print(idx)
    print(f"{x}: {len(os.listdir(f'{projectPath}/{x}'))}")

0
normal: 57
1
osteopenia: 241
2
osteoporosis: 73


In [41]:
from PIL import Image
imageSizes = {}
for x in pathList:
    img = Image.open(x).size
    try:
        imageSizes[str(img)] = imageSizes[str(img)] + 1
    except KeyError:
        imageSizes[str(img)] = 1
imageSizes #varied image sizes, have to resize to 1024,1024

{'(244, 244)': 371}

In [42]:
import torchvision
import torch
import torchvision.transforms.functional as vF


class OsteoTorchDataset(torch.utils.data.Dataset):
    def __init__(self, itemsPath:list, labels:list, transform=None, std = False, mean = False): #mean on if mean needs to be scaled, same goes for std
        
        self.itemsPath = itemsPath
        self.labels = labels
        self.transform = transform
        self.std = std
        self.mean = mean

    def __len__(self):
        return len(self.itemsPath)

    def __getitem__(self,idx)->tuple[Image.Image,int]:
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        image = Image.open(self.itemsPath[idx]).convert('RGB')
        # image = torchvision.transforms.functional.pil_to_tensor(image)

        if self.transform:
            image = self.transform(image)  
        
        image.to(device = torch.device('cuda'))

        if self.mean or self.std:
            image = self.scaler(image,self.mean,self.std)
        

        return image, self.labels[idx], self.itemsPath[idx]          
    
    def scaler(self,image:torch.tensor,normMean = False,normStd = False): #TODO better way to do this? also huge bottleneck without cuda
        imageSum = torch.sum(image,dim=(1,2))
        mean = imageSum/(image.shape[1]*image.shape[2])
        sum_squared_tensor = torch.sum((imageSum - mean)**2)
        std = torch.sqrt(sum_squared_tensor/image.shape[1]*image.shape[2])

        if not normMean:
            mean = torch.zeros_like(mean,device='cuda')

        if not normStd:
            std = torch.ones_like(std,device='cuda')

        image = torchvision.transforms.functional.normalize(image.float(),mean,std)
        return image

transform = torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Resize((256,256)),
                                            torchvision.transforms.CenterCrop((224,224)),
                                            torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                                            # ,torchvision.transforms.Grayscale()
                                            ])
osteoDataset = OsteoTorchDataset(pathList,labelList,transform,std=True,mean=True)                                  

In [43]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import WeightedRandomSampler

osteoDataset = OsteoTorchDataset(pathList,labelList,transform,std=False,mean=False)      
train,val = torch.utils.data.random_split(osteoDataset,[0.8,0.2])#MAY BUG
trainLabels = [y for x,y,z in train]
unique_elements, counts = torch.unique(torch.tensor(trainLabels), return_counts=True)
sampleWeights = 1. / counts.float() #I HAVE NO IDEA WHY THIS WORKS BUT IT DOES
# MORAL OF THE STORY: FOLLOW THE FUCKING TUTORIAL DONT TRY CHANGING SHIT ON YOUR OWN ***EVEN IF THE DOCUMENTATION SAYS YOU SHOULD***
trainSampler = WeightedRandomSampler(weights=[sampleWeights[x] for x in trainLabels],num_samples=len(train),replacement=True)

trainLoader = DataLoader(train, batch_size = 8,shuffle=False,num_workers=0,sampler=trainSampler)
valLoader = DataLoader(val, batch_size = 8,shuffle=False,num_workers=0)

unique_elements, counts = torch.unique(torch.cat([x for y,x,z in trainLoader]), return_counts=True)#delet this
counts

tensor([101,  89, 107])

In [44]:
unique_elements, counts = torch.unique(torch.cat([x for y,x,z in valLoader]), return_counts=True)#delet this
counts

tensor([10, 48, 16])

In [45]:
temp = next(iter(trainLoader))

In [46]:
temp[0][0].max()

tensor(0.7453)

In [47]:
index = -1

In [56]:
# temp[0][0].min()
import matplotlib.pyplot as plt
index += 1
# print(temp[1][index],temp[2][index])
print((temp[0][index].numpy()*255).max().astype(np.uint8))

fig, axs = plt.subplots(1, 2)

print(str(temp[2][index]))
# Plot the first image
axs[0].imshow(((temp[0][index]+1)/2).permute(1, 2, 0))
axs[0].set_title('Image 1')

# Plot the second image
axs[1].imshow(Image.open(temp[2][index]).convert('RGB'))
axs[1].set_title('Image 2')

plt.show()

IndexError: index 8 is out of bounds for dimension 0 with size 8

In [ ]:
# import models here
from models import AlexNet,resnetWrapper
from resnetModel import ResNet50
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from torcheval.metrics.functional import multiclass_f1_score,multiclass_confusion_matrix,multiclass_accuracy
from torchmetrics.classification import MulticlassConfusionMatrix
from matplotlib import pyplot as plt
from torch import mode
import torch.nn.functional as F
import itertools
import seaborn as sns

#FIXME identify bottleneck
#FIXME clean up my fucking code ffs its so UGLY

class ExperimentModel(L.LightningModule):

    existingModels = []

    def __init__(self,pretrainedModel,resize = False,num_class = 3,outputSize = 1000,pretrained = False) -> None:
        ExperimentModel.existingModels.append(self)
        super().__init__()
        if type(pretrainedModel) != str:
            print("using model as-is")
            self.model = pretrainedModel
        else:
            self.model = torch.hub.load('pytorch/vision:v0.19.0', pretrainedModel, pretrained=pretrained)
            
            
        # if pretrainedPreprocess:
        #     self.preprocess = pretrainedPreprocess  
        # for param in self.model.parameters():
        #     param.requires_grad = False
        self.resize = resize

        if self.resize:
            linear1 = None
            try: 
                linear1 = torch.nn.Linear(self.model.fc.out_features, 256)
            except AttributeError:
                print("no fc detected, switcing to provided outputSize")
                linear1 = torch.nn.Linear(outputSize, 256)
            self.customHead = torch.nn.Sequential(linear1,torch.nn.ReLU(),torch.nn.Linear(256,num_class))

        self.num_class = num_class
        self.criterion = torch.nn.CrossEntropyLoss()

        self.valLog = []
        self.epoch = []
        self.valPreds = []
        self.valLabels = []
        self.valScore = []

        self.bestValPreds = [[]]
        self.bestValLabels = [[]]
        self.bestValScore = []

        self.dump = []
        
    def forward(self,input):
        out = self.model(input.float())
        if self.resize:
            out = self.customHead(out)
        return out
        
    def training_step(self,batch):
        data,label,path = batch

        output = self(data)
        # print('\n train pred vs label')
        # print(output.argmax(1))
        # print(data[0])
        # print(label[0])
        self.dump.append([path,label])
        loss = self.criterion(output,label)
        # print(loss)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx) -> None:
        data,label,path = batch
        output = self(data)

        preds = output.argmax(1)
        loss = F.cross_entropy(output,label)
        self.valLabels.append(label)
        self.valPreds.append(preds)
        self.valScore.append(multiclass_f1_score(preds,label,num_classes=self.num_class,average = 'macro'))
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc_F1", multiclass_f1_score(preds,label,num_classes=self.num_class,average = 'macro'), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc", multiclass_accuracy(preds,label,num_classes=self.num_class), on_step=False, on_epoch=True, prog_bar=True, logger=True)

    
    def on_validation_epoch_end(self) -> None:
        if len(self.valScore) == 2:
            return None
        
        accuracy = torch.stack(self.valScore).mean()
        self.bestValScore.append(accuracy)
        self.bestValPreds.append(torch.cat(self.valPreds))
        self.bestValLabels.append(torch.cat(self.valLabels))
        print(f"preds: {torch.cat(self.valPreds)}\n labels: {torch.cat(self.valLabels)}")
        # self.log("val_acc_F1", multiclass_f1_score(torch.cat(self.valPreds),torch.cat(self.valLabels),num_classes=self.num_class,average = 'macro'), on_step=False, on_epoch=True, prog_bar=True, logger=True)
       
        # print(f"\n Validation accuracy: {accuracy}")
        # print(f"bestValScore: {self.bestValScore}")
        self.valPreds = []
        self.valLabels = []
        self.valScore = []
        

    def configure_optimizers(self):
        return torch.optim.Adagrad(self.parameters(), lr=0.005)

    def predict_step(self, batch):
        data,label = batch
        return self(data.float())

# class ResNetCallbacks(L.Callback): #TODO implement callbacks
#     def on_validation_epoch_end(trainer, pl_module):
#         valLog.append(sum(self.cumLog)/len(self.cumlog))

# alexNet = ExperimentAlexNet(AlexNet(num_classes=3))
# resNet = ExperimentAlexNet(ResNet50(num_classes=3))
preResNet = ExperimentModel(ResNet50(num_classes=3),num_class=3)
resnet50 = ExperimentModel("resnet50",resize=True,pretrained=True,num_class=3)
vgg19 = ExperimentModel(model,resize=False,pretrained=False,num_class=3)
effnet = ExperimentModel("efficientnet_v2_m",resize=True,pretrained=True,num_class=3)
mobilenet = ExperimentModel("mobilenet_v3_large",resize=True,pretrained=True,num_class=3)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.19.0', "vgg19",{"num_classes":3}, pretrained=True, )
model.classifier[6] = torch.nn.Linear(4096,3)
model

In [ ]:
ExperimentModel.existingModels[0].model.__class__.__name__

In [ ]:
# import re
# labelProcess = [re.search(r'\/.*\/',x)[0] for x in preResNet.dump[0][0]]
# [x.strip('\/') for x in labelProcess]

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
for x in ExperimentModel.existingModels:
    gc.collect()
    torch.cuda.empty_cache()
    # early_stopping = EarlyStopping(
    #     monitor='val_loss',  # Metric to monitor
    #     patience=5000,          # Number of epochs with no improvement after which training will be stopped
    #     verbose=False,        # Verbosity mode
    #     mode='min'           # Mode can be 'min', 'max', or 'auto'
    # )

    logger = TensorBoardLogger("tb_logs", name=f"{x.model.__class__.__name__}")


    trainer = L.Trainer(max_epochs = 75,accelerator='gpu', devices='auto', precision='16-mixed',logger=logger)
    trainer.fit(model=x,train_dataloaders=trainLoader,val_dataloaders=valLoader)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# [[y.argmax().item() for y in w] for w in x.bestValPreds]
# ([1, 2, 0, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 1,
#          1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1]
# [[y.argmax().item() for y in w] for w in x.bestValPreds]
preds = torch.tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 0, 2, 2, 1, 1,
        1, 1])
labels = torch.tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 1, 2, 2, 1, 2, 1, 1, 1,
        1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 0, 0, 2, 2, 0, 1, 1, 2, 2, 1, 1, 0, 2,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1])
index = 50
cm = confusion_matrix(preds,labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
# plt.plot(confusion_matrix([[y.argmax().item() for y in w] for w in x.bestValPreds][index],x.bestValLabels[1].cpu()))
print(classification_report(preds,labels))


In [ ]:
# raise AssertionError("Stop here")
# alex net acc F1 = 0.62 loss = 0.923
# resnet18 acc F1 = ? loss = 1
# resnet50 acc F1 = 0.29 loss = 3
# resnet50 Custom acc F1 = 0.62 loss = 0.957
# resnet50 Custom acc oversampled F1 = 0.78 loss = 0.759
# mobilenet 
# effnet
# Vits 

# model = alexNetNonMod.load_from_checkpoint("/path/to/checkpoint.ckpt")

In [ ]:
model = torch.hub.load('pytorch/vision:v0.19.0', "vgg19",{"num_classes":3}, pretrained=True, )
model.classifier[6] = torch.nn.Linear(4096,3)
model

In [ ]:
import torch

import torch.nn as nn
import torch.optim as optim
torch.cuda.empty_cache()
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Set the device to GPU if available
device = torch.device("cuda")

# Move the model to the device
model = model.to(device)

# Set the model to training mode
model.train()

num_epochs = 50

accHistory = []
lossHistory = []
valAccHistory = []
valLabs = []
valPreds = []
print("Starting training...")

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_accuracy = []
    
    # Iterate over the training dataset
    for inputs, labels, _ in trainLoader:
        # Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs.float())
        predicted_labels = torch.argmax(outputs, dim=1)
        accuracy = torch.sum(predicted_labels == labels).item() / labels.size(0)
        running_accuracy.append(accuracy)
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
    
    # Print the average loss for the epoch
    print(f"Epoch {epoch+1} - Loss: {running_loss / len(trainLoader)}")
    print(F"Accuracy: {sum(running_accuracy)/len(running_accuracy)}")
    accHistory.append(sum(running_accuracy)/len(running_accuracy))
    lossHistory.append(running_loss / len(trainLoader))

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        
        runningValAccHistory = []
        runningValLabs = []
        runningValPreds = []
        for inputs, labels, _ in valLoader:

            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs.float())

            # Compute the loss
            loss = criterion(outputs, labels)

            # Compute the predicted labels
            predicted_labels = torch.argmax(outputs, dim=1)

            # Compute the accuracy
            accuracy = torch.sum(predicted_labels == labels).item() / labels.size(0)

            runningValAccHistory.append(accuracy)
            runningValLabs.append(labels)
            runningValPreds.append(predicted_labels)

        # print(f"Validation Loss: {loss.item()}")
        valPreds.append(runningValPreds)
        valLabs.append(runningValLabs)
        print(f"Validation Accuracy: {sum(runningValAccHistory)/len(valLoader)}")
        valAccHistory.append(sum(runningValAccHistory)/len(valLoader))

In [ ]:
valAccHistory[-5]
predsTest = torch.cat(valPreds[48])
labelsTest = torch.cat(valLabs[48])
print(classification_report(predsTest.cpu(),labelsTest.cpu()))
cm= confusion_matrix(predsTest.cpu(),labelsTest.cpu(),normalize='true')
sns.heatmap(cm, annot=True, cmap='Blues', fmt='.2f',)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
labelsTest

In [ ]:
plt.plot(valAccHistory)

In [ ]:
input , label, _ = next(iter(valLoader))
# Move the inputs and labels to the device
inputs = inputs.to(device)
labels = labels.to(device)

# Forward pass
outputs = model(inputs)
predicted_labels = torch.argmax(outputs, dim=1)
print(predicted_labels)


In [ ]:
import seaborn as sns
import pandas as pd

# Create a dataframe with the data
df = pd.DataFrame({'Epoch': range(len(accHistory)), 'Accuracy': accHistory, 'Loss': lossHistory, 'Validation Accuracy': valAccHistory})

# Create the line plot
sns.relplot(data=df, x='Epoch', y='Accuracy', kind='line')
sns.relplot(data=df, x='Epoch', y='Loss', kind='line')
sns.relplot(data=df, x='Epoch', y='Validation Accuracy', kind='line')

In [ ]:
torch.save(model.state_dict(), "resnetClassic.pth")


In [ ]:
outputs = []
label = []
import gc
gc.collect()
torch.cuda.empty_cache()
with torch.no_grad():
    for inputs, labels, _ in valLoader:
        # Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        label = labels
        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Compute the predicted labels
        predicted_labels = torch.argmax(outputs, dim=1)

        # Compute the accuracy
        accuracy = torch.sum(predicted_labels == labels).item() / labels.size(0)

In [ ]:
print(outputs.argmax(1))
print(label)